In [1]:
import numpy as np
import math
import sys
from collections import defaultdict
from math import expm1
import time

In [2]:
def computeInitialProb(trainDataFile,numOfStates):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    headerLine = metaDataLine.split(" ")
    numSequences = int(headerLine[0])
    distinctObservations= int(headerLine[1])#Total Number of Distinct Observations
    numOfStates=min(numOfStates,distinctObservations)
    empiricalCount=np.zeros(shape=numOfStates)
    empiricalFreq=defaultdict(int)
    for n in range(numSequences):
        line = trainFile.readline()#Reading Sequences 1 by 1
        l = line.split(" ")
        startState=int(l[1])
        empiricalFreq[startState] = empiricalFreq[startState]+1
    totalObservations=0
    for i in np.arange(numOfStates):
        empiricalCount[i]=empiricalFreq[i]
        totalObservations=totalObservations+empiricalCount[i]
    initialProb=[count/totalObservations for count in empiricalCount]
    return (numOfStates,distinctObservations,initialProb)

In [3]:
def createRandomMatrixA(numOfStates):
    matrixA=np.zeros(shape=(numOfStates,numOfStates),dtype=float)
    prob=1.0/(numOfStates*numOfStates)
    for i in np.arange(numOfStates):
        for j in np.arange(numOfStates):
            matrixA[i][j]=prob
    return matrixA
def createRandomMatrixB(numOfStates,distinctObservations):
    matrixB=np.zeros(shape=(numOfStates,distinctObservations),dtype=float)
    prob=1.0/(numOfStates*distinctObservations)
    for i in np.arange(numOfStates):
        for j in np.arange(distinctObservations):
            matrixB[i][j]=prob
    return matrixB

In [4]:
def computeAlpha(observations,a,b,pi,alphaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    alphaDpScaleTime0=0    
    for i in np.arange(statesC):
        alphaDP[0][i]=pi[i]*b[i][observations[0]]
        alphaDpScaleTime0+=alphaDP[0][i]
    for i in np.arange(statesC):
        alphaDP[0][i]/=alphaDpScaleTime0
    #print("Initial Alpha ",alphaDP[0])
    for t in np.arange(1,timePts):
        alphaDpScaleTimeT=0
        for i in np.arange(statesC):
            for j in np.arange(statesC):
                alphaDP[t][i]+=alphaDP[t-1][j]*a[j][i]
            alphaDP[t][i]*=b[i][observations[t]]
            alphaDpScaleTimeT+=alphaDP[t][i]
        #print("Scaling ",alphaDpScaleTimeT)
        for i in np.arange(statesC):
            alphaDP[t][i]/=alphaDpScaleTimeT
    #print("Next AlphaDP ",alphaDP[1])
    #print("alphaDP ",alphaDP)
def observationsLikelihood(alphaDP):
    timePts=alphaDP.shape[0]
    stateC=alphaDP.shape[1]
    ans=0.0
    for i in np.arange(stateC):
        ans+=alphaDP[timePts-1][i]
        #print("Alpha@Timept ",i ," : ", alphaDP[i])
    #print("Observation Likelihood ",ans)
    return ans

In [5]:
"""β[t][i]: Probability of seeing the observations from t + 1 to end(T), given that we are in state i at time t
beta[t][i]= a[i][0]*b[0][o[t+1]]*beta[t+1][0] + a[i][1]*b[1][o[t+1]]*beta[t+1][1] + ......... +
                a[i][N-2]*b[N-2][o[t+1]]*beta[t+1][N-2] + a[i][N-1]*b[N-1][o[t+1]]*beta[t+1][N-1]"""
def computeBeta(observations,a,b,pi,betaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    for state in np.arange(statesC):
            betaDP[timePts-1][state]=1
    for t in np.arange(timePts-2,-1,-1):
        betaDpScaleTimeT=0
        for i in np.arange(statesC):
            for j in np.arange(statesC):
                betaDP[t][i]+=a[i][j]*b[j][observations[t+1]]*betaDP[t+1][j]
            betaDpScaleTimeT+=betaDP[t][i]
        #if t==timePts-2:
            #print("Non Scaled T-2 ",betaDP[t])
        #print("Scale At ",t," ",betaDpScaleTimeT)
        for i in np.arange(statesC):
            betaDP[t][i]/=betaDpScaleTimeT
    #print("BetaDP t-2 ",betaDP[timePts-2])
    #print("betaDP ",betaDP)
    return betaDP

In [6]:
def computeDiGammaNum(t,i,j,alphaDP,betaDP,a,b,observations):
    return alphaDP[t][i]*a[i][j]*b[j][observations[t+1]]*betaDP[t+1][j]
def computeDiGammaDP(alphaDP,betaDP,a,b,observations):
    observationsC=alphaDP.shape[0]
    statesC=alphaDP.shape[1]
    diGammaDP=np.zeros(shape=(statesC,statesC),dtype=float)
    diGammaDenom=observationsLikelihood(alphaDP)
    #print("Observation Likelihood ",diGammaDenom ," ",observations)
    #print("Observation ",observations)
    #print("DiGammaDenom ",diGammaDenom ,"Observation : ",observations,"AlphaDP ",alphaDP)
    for i in np.arange(statesC):
        for j in np.arange(statesC):
                for t in np.arange(observationsC-1):
                    #print("DiGama ",i," ",j," ",t," ",computeDiGammaNum(t,i,j,alphaDP,betaDP,a,b,observations))
                    diGammaDP[i][j]+=computeDiGammaNum(t,i,j,alphaDP,betaDP,a,b,observations)/diGammaDenom
    #print("diGammaDP ",diGammaDP)
    return diGammaDP
def computeTransitionProbabilityA(alphaDP,betaDP,a,b,observations):
    #print("Got A ",a)
    statesC=alphaDP.shape[1]
    newlyComputedTransitionProbA=np.zeros(shape=(statesC,statesC),dtype=float)
    diGammaDP=computeDiGammaDP(alphaDP,betaDP,a,b,observations)
    #print("DiGamma ",diGammaDP)
    diGammaDPSumGrpByJ=np.zeros(shape=(statesC),dtype=float)
    #print("DiGammaDP ",diGammaDP)
    for i in np.arange(statesC):
        sumAcrossJ=0.0
        for j in np.arange(statesC):
            sumAcrossJ+=diGammaDP[i][j]
        diGammaDPSumGrpByJ[i]=sumAcrossJ
    #print("DiGramGrpByJ ",diGammaDPSumGrpByJ)
    for i in np.arange(statesC):    
        for j in np.arange(statesC):
            if (diGammaDPSumGrpByJ[i]==0):
                #print("Underflow ",i," ",diGammaDPSumGrpByJ[i])
                newlyComputedTransitionProbA[i][j]=0.0
            else:
                newlyComputedTransitionProbA[i][j]=diGammaDP[i][j]/diGammaDPSumGrpByJ[i]
    #print("Updated A ",newlyComputedTransitionProbA)
    return newlyComputedTransitionProbA   

In [7]:
def computeGammaNum(t,j,alphaDP,betaDP):
    return alphaDP[t][j]*betaDP[t][j]
def computeGammaDP(alphaDP,betaDP):
    observationsC=alphaDP.shape[0]
    statesC=alphaDP.shape[1]
    gammaDP=np.zeros(shape=(statesC,observationsC),dtype=float)
    gammaDenom=observationsLikelihood(alphaDP)
    for i in np.arange(statesC):
        for t in np.arange(observationsC):
            gammaDP[i][t]=computeGammaNum(t,i,alphaDP,betaDP)/gammaDenom  
    return gammaDP
def computeObsrProbNum(gammaDP,i,vk,observations):
    observationC=len(observations)
    obsrProbNum=0.0
    for t in np.arange(observationC):
        if observations[t]==vk:
            obsrProbNum+=gammaDP[i][t]
    return obsrProbNum
def computeTransitionProbabilityB(alphaDP,betaDP,a,b,observations,observationDict):
    statesC=a.shape[0]
    observationsC=b.shape[1]
    newlyComputedObsrProbB=np.zeros(shape=(statesC,observationsC),dtype=float)
    gammaDP=computeGammaDP(alphaDP,betaDP) 
    #print("gammaDP ",gammaDP)
    #print("ALPHADP : ",alphaDP)
    #print("BETADP : ",betaDP)
    for i in np.arange(statesC):
        obsrProbDenom =np.sum(gammaDP[i])
        for vk in observationDict:
            newlyComputedObsrProbB[i][vk]=computeObsrProbNum(gammaDP,i,vk,observations)/obsrProbDenom
    return newlyComputedObsrProbB

In [16]:
#Change Convergence Criteria to be more reasonable/Useful
def isConverged(count):
    if count>=7:
        return True
    return False
def Forward_Backward_EM_Algo(observations,A,B,pi,observationDict):
    count=0
    updatedA=A
    updatedB=B
    while isConverged(count)==False:
        #Expectation(E)-Step
        alphaDP=np.zeros(shape=(observations.shape[0],updatedA.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        betaDP=np.zeros(shape=(observations.shape[0],updatedA.shape[0]))# Count_of_Observations*Count_of_Hidden_States
        computeAlpha(observations,updatedA,updatedB,pi,alphaDP)
        #print("AlphaDP Computed ....")
        #print("ALPHA-DP ",alphaDP)
        computeBeta(observations,updatedA,updatedB,pi,betaDP)
        #Maximization(M)-Step
        newA=computeTransitionProbabilityA(alphaDP,betaDP,updatedA,updatedB,observations)
        newB=computeTransitionProbabilityB(alphaDP,betaDP,updatedA,updatedB,observations,observationDict)
        #print("New A =================================>")
        #print(newA)
        #print("New B =================================>")
        #print(newB)
        updatedA=newA
        updatedB=newB
        count=count+1
    return (updatedA,updatedB)

In [17]:
def trainHMM(trainDataFile,A,B,pi,maxSequences=-1):
    trainFile=open(trainDataFile,"r")
    metaDataLine = trainFile.readline()
    headerLine = metaDataLine.split(" ")
    numSequences = int(headerLine[0])
    distinctObservations= int(headerLine[1])#Total Number of Distinct Observations
    observationDict=np.arange(distinctObservations)
    updatedA=np.NaN
    updatedB=np.NaN
    isAUpdated=False
    #for n in range(numSequences):
    if(maxSequences==-1):
        usedSeqs=numSequences
    else:
        usedSeqs=min(maxSequences,numSequences)
    actuallyUsedSeqs=0
    for n in range(usedSeqs):
        line = trainFile.readline()#Reading Sequences 1 by 1
        l = line.split(" ")
        #print("For Sequence ",n," =====================================>")
        if(int(l[0])<=1):
            #print("Skipping ",l)
            continue
        actuallyUsedSeqs+=1
        observations=np.array([int(i) for i in l[1:len(l)]])
        learnedParams=Forward_Backward_EM_Algo(observations,A,B,pi,observationDict)
        if isAUpdated==False:
            isAUpdated=True
            updatedA=learnedParams[0]
            updatedB=learnedParams[1]
        else:
            updatedA+=learnedParams[0]
            updatedB+=learnedParams[1]
        #print("State Transition Matrix (A) ===>")
        #print(learnedParams[0])
        #print("Observation Probability Matrix (B) ===>")
        #print(learnedParams[1])
        #print("Aggregated State Transition Matrix (A) ===>")
        #print(updatedA)
        #print("Aggregated Observation Probability Matrix (B) ===>")
        #print(updatedB)
    updatedA=updatedA/actuallyUsedSeqs
    updatedB=updatedB/actuallyUsedSeqs
    return (updatedA,updatedB)

In [18]:
def trainModel(fileLoc,maxNoOfStates,maxSequences=-1):
    start = time.time()
    initialProbs=computeInitialProb(fileLoc,maxNoOfStates)
    end = time.time()
    print("Computed Initial Prob. in ", end - start ,"seconds")
    pi=initialProbs[2]
    numOfStates=initialProbs[0]
    distinctObservations=initialProbs[1]
    #print(initialProbs)
    A=createRandomMatrixA(numOfStates)
    B=createRandomMatrixB(numOfStates,distinctObservations)
    #print(A)
    #print(B)
    trainedParams=trainHMM(fileLoc,A,B,pi,maxSequences)
    end=time.time()
    #print("For ",maxSequences," Sequences : Total Training Time ",end-start)
    return trainedParams

In [19]:
(A,B)=trainModel('Data/1.spice.train.txt',20,1)

Computed Initial Prob. in  0.16648101806640625 seconds


In [25]:
%timeit trainModel('Data/1.spice.train.txt',20,15)
#(A,B)

Computed Initial Prob. in  0.24859118461608887 seconds
Computed Initial Prob. in  0.26802635192871094 seconds
Computed Initial Prob. in  0.36754536628723145 seconds
Computed Initial Prob. in  0.25893568992614746 seconds
1 loops, best of 3: 45.8 s per loop


In [40]:
(A,B)=trainModel('Data/1.spice.train.txt',20,150)

Computed Initial Prob. in  0.21947813034057617 seconds
(20, 20, [0.036700000000000003, 0.0091999999999999998, 0.020549999999999999, 0.081900000000000001, 0.042299999999999997, 0.04845, 0.11260000000000001, 0.0385, 0.018249999999999999, 0.045999999999999999, 0.016750000000000001, 0.042049999999999997, 0.049799999999999997, 0.065449999999999994, 0.06855, 0.1487, 0.028850000000000001, 0.021999999999999999, 0.0591, 0.044299999999999999])
For  150  Sequences : Total Training Time  513.4503910541534


In [44]:
(A,B)=trainModel('Data/1.spice.train.txt',20,1500)

Computed Initial Prob. in  0.17683792114257812 seconds
(20, 20, [0.036700000000000003, 0.0091999999999999998, 0.020549999999999999, 0.081900000000000001, 0.042299999999999997, 0.04845, 0.11260000000000001, 0.0385, 0.018249999999999999, 0.045999999999999999, 0.016750000000000001, 0.042049999999999997, 0.049799999999999997, 0.065449999999999994, 0.06855, 0.1487, 0.028850000000000001, 0.021999999999999999, 0.0591, 0.044299999999999999])
For  1500  Sequences : Total Training Time  6131.2775712013245


In [43]:
#(A,B)=trainModel('Data/1.spice.train.txt',20,1600)
#(A,B)